this is raw code to start with

In [1]:
#define data path
data_path=r"/content/drive/MyDrive/dan_brown.txt"

In [2]:
!pip install chardet

In [4]:
import numpy as np
import chardet
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import tensorflow as tf
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import io
import requests
import re
import torch
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
#this functon to read data set and return list with lines of data rows
def read_text_dataset(filename, encodings=['utf-8', 'latin-1']):
    dataset = []
    for encoding in encodings:
        try:
            with open(filename, 'r', encoding=encoding) as file:
                for line in file:
                    line = line.strip()
                    if line:
                        dataset.append(line)
            return dataset
        except UnicodeDecodeError:
            pass
    raise ValueError("Unable to decode the file using the provided encodings")

In [6]:
#call the func
dataset = read_text_dataset(data_path,)
print(dataset)


['The Da Vinci Code Dan Brown   FOR BLYTHE... AGAIN. MORE THAN EVER.', 'Acknowledgments', 'First and foremost, to my friend and editor, Jason Kaufman, for working so hard on this project and', 'for truly understanding what this book is all about. And to the incomparable Heide Lange--tireless', 'champion of The Da Vinci Code, agent extraordinaire, and trusted friend.', 'I cannot fully express my gratitude to the exceptional team at Doubleday, for their generosity, faith,', 'and superb guidance. Thank you especially to Bill Thomas and Steve Rubin, who believed in this', 'book from the start. My thanks also to the initial core of early in-house supporters, headed by', 'Michael Palgon, Suzanne Herz, Janelle Moburg, Jackie Everly, and Adrienne Sparks, as well as to', "the talented people of Doubleday's sales force.", 'For their generous assistance in the research of the book, I would like to acknowledge the Louvre', 'Museum, the French Ministry of Culture, Project Gutenberg, Bibliothèque Na

In [7]:
print(len(dataset))
print(np.array(dataset).shape)

10959
(10959,)


In [8]:
# second way to read the data set  and return the text not List like the previos way
def read_text_file_second_way(filename):
    with open(filename, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        file_contents = raw_data.decode(encoding)
    return file_contents



In [9]:
data= read_text_file_second_way(data_path)

In [10]:
data

'\x0c The Da Vinci Code Dan Brown   FOR BLYTHE... AGAIN. MORE THAN EVER.\n\nAcknowledgments\n\nFirst and foremost, to my friend and editor, Jason Kaufman, for working so hard on this project and\nfor truly understanding what this book is all about. And to the incomparable Heide Lange--tireless\nchampion of The Da Vinci Code, agent extraordinaire, and trusted friend.\n\nI cannot fully express my gratitude to the exceptional team at Doubleday, for their generosity, faith,\nand superb guidance. Thank you especially to Bill Thomas and Steve Rubin, who believed in this\nbook from the start. My thanks also to the initial core of early in-house supporters, headed by\nMichael Palgon, Suzanne Herz, Janelle Moburg, Jackie Everly, and Adrienne Sparks, as well as to\nthe talented people of Doubleday\'s sales force.\n\nFor their generous assistance in the research of the book, I would like to acknowledge the Louvre\nMuseum, the French Ministry of Culture, Project Gutenberg, Bibliothèque Nationale, 

In [11]:
# see the words in the data
words_in_my_data=data.split()
words_in_my_data

['The',
 'Da',
 'Vinci',
 'Code',
 'Dan',
 'Brown',
 'FOR',
 'BLYTHE...',
 'AGAIN.',
 'MORE',
 'THAN',
 'EVER.',
 'Acknowledgments',
 'First',
 'and',
 'foremost,',
 'to',
 'my',
 'friend',
 'and',
 'editor,',
 'Jason',
 'Kaufman,',
 'for',
 'working',
 'so',
 'hard',
 'on',
 'this',
 'project',
 'and',
 'for',
 'truly',
 'understanding',
 'what',
 'this',
 'book',
 'is',
 'all',
 'about.',
 'And',
 'to',
 'the',
 'incomparable',
 'Heide',
 'Lange--tireless',
 'champion',
 'of',
 'The',
 'Da',
 'Vinci',
 'Code,',
 'agent',
 'extraordinaire,',
 'and',
 'trusted',
 'friend.',
 'I',
 'cannot',
 'fully',
 'express',
 'my',
 'gratitude',
 'to',
 'the',
 'exceptional',
 'team',
 'at',
 'Doubleday,',
 'for',
 'their',
 'generosity,',
 'faith,',
 'and',
 'superb',
 'guidance.',
 'Thank',
 'you',
 'especially',
 'to',
 'Bill',
 'Thomas',
 'and',
 'Steve',
 'Rubin,',
 'who',
 'believed',
 'in',
 'this',
 'book',
 'from',
 'the',
 'start.',
 'My',
 'thanks',
 'also',
 'to',
 'the',
 'initial',
 '

start preprocessing

In [12]:
# this function to return list of prepocessed dataset
def preprocess_text_dataset(dataset):
    # Download stopwords if not already downloaded
    nltk.download('stopwords')

    # Initialize stopwords
    stop_words = set(stopwords.words('english'))

    preprocessed_dataset = []
    for text in dataset:
        # Remove non-alphabetic characters and convert to lowercase
        text = re.sub(r'[^a-zA-Z]', ' ', text).lower()

        # Tokenize the text
        tokens = word_tokenize(text)

        # Remove stopwords
        tokens = [token for token in tokens if token not in stop_words]

        # Join the tokens back into a single string
        preprocessed_text = ' '.join(tokens)

        preprocessed_dataset.append(preprocessed_text)
    return preprocessed_dataset



In [13]:
preprocessed_data = preprocess_text_dataset(dataset)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
print(preprocessed_data[0])
print(dataset[0])



da vinci code dan brown blythe ever
The Da Vinci Code Dan Brown   FOR BLYTHE... AGAIN. MORE THAN EVER.


In [15]:
data = ''.join(preprocessed_data)
print(data)

da vinci code dan brown blythe everacknowledgmentsfirst foremost friend editor jason kaufman working hard projecttruly understanding book incomparable heide lange tirelesschampion da vinci code agent extraordinaire trusted friendfully express gratitude exceptional team doubleday generosity faithsuperb guidance thank especially bill thomas steve rubin believedbook start thanks also initial core early house supporters headedmichael palgon suzanne herz janelle moburg jackie everly adrienne sparks welltalented people doubleday sales forcegenerous assistance research book would like acknowledge louvremuseum french ministry culture project gutenberg biblioth que nationale gnosticsociety library department paintings study documentation service louvrecatholic world news royal observatory greenwich london record society munimentcollection westminster abbey john pike federation american scientistsfive members opus dei three active two former recounted stories positivenegative regarding experienc

In [16]:
#this function for custom preprocessing
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
  stripped_num  = tf.strings.regex_replace(stripped_html, "[\d-]", " ")
  stripped_punc  =tf.strings.regex_replace(stripped_num,"[%s]" % re.escape(string.punctuation), "")
  return stripped_punc

In [17]:
data=custom_standardization(data)

In [18]:
data

<tf.Tensor: shape=(), dtype=string, numpy=b'da vinci code dan brown blythe everacknowledgmentsfirst foremost friend editor jason kaufman working hard projecttruly understanding book incomparable heide lange tirelesschampion da vinci code agent extraordinaire trusted friendfully express gratitude exceptional team doubleday generosity faithsuperb guidance thank especially bill thomas steve rubin believedbook start thanks also initial core early house supporters headedmichael palgon suzanne herz janelle moburg jackie everly adrienne sparks welltalented people doubleday sales forcegenerous assistance research book would like acknowledge louvremuseum french ministry culture project gutenberg biblioth que nationale gnosticsociety library department paintings study documentation service louvrecatholic world news royal observatory greenwich london record society munimentcollection westminster abbey john pike federation american scientistsfive members opus dei three active two former recounted 

the previos corpus will be used after some preprocessing yet

now we will start with some preprocessing to train one model

In [19]:
data1= read_text_file_second_way(data_path)

In [20]:
raw_text=data1

In [21]:
processed_text = raw_text.lower()
processed_text = re.sub(r'[^\x00-\x7f]',r'', processed_text)

print('corpus length:', len(processed_text))

chars = sorted(list(set(processed_text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 830543
total chars: 54


In [22]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(processed_text) - maxlen, step):
    sentences.append(processed_text[i: i + maxlen])
    next_chars.append(processed_text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 276835


In [23]:
sentences

['\x0c the da vinci code dan brown   for blyt',
 'he da vinci code dan brown   for blythe.',
 'da vinci code dan brown   for blythe... ',
 'vinci code dan brown   for blythe... aga',
 'ci code dan brown   for blythe... again.',
 'code dan brown   for blythe... again. mo',
 'e dan brown   for blythe... again. more ',
 'an brown   for blythe... again. more tha',
 'brown   for blythe... again. more than e',
 'wn   for blythe... again. more than ever',
 '  for blythe... again. more than ever.\n\n',
 'or blythe... again. more than ever.\n\nack',
 'blythe... again. more than ever.\n\nacknow',
 'the... again. more than ever.\n\nacknowled',
 '... again. more than ever.\n\nacknowledgme',
 ' again. more than ever.\n\nacknowledgments',
 'ain. more than ever.\n\nacknowledgments\n\nf',
 '. more than ever.\n\nacknowledgments\n\nfirs',
 'ore than ever.\n\nacknowledgments\n\nfirst a',
 ' than ever.\n\nacknowledgments\n\nfirst and ',
 'an ever.\n\nacknowledgments\n\nfirst and for',
 'ever.\n\nacknowled

In [24]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


<ipython-input-24-b1fdc504c468>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
<ipython-input-24-b1fdc504c468>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


### finish preprocessing and start modeling

In [26]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               93696     
                                                                 
 dense (Dense)               (None, 54)                6966      
                                                                 
Total params: 100,662
Trainable params: 100,662
Non-trainable params: 0
_________________________________________________________________


In [28]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [29]:

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print("******************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(processed_text) - maxlen - 1)
    for temperature in [1.0]:
        print('----- temperature:', temperature)

        generated = ''
        sentence = processed_text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

#fit the model for 10 epochs

In [30]:

import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Fit the model
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=10,
          callbacks=[print_callback])

Epoch 1/10
2162/2163 [============================>.] - ETA: 0s - loss: 2.4915******************************************************
----- Generating text after Epoch: 0
----- temperature: 0.2
----- Generating with seed: "ou not yet understand?" he paused. "we m"
ou not yet understand?" he paused. "we meare the the the the the the serer the the the the the the the the the the the sout at in the the the che the the pher the the the the the the the the the the the the the the the sered an the mang the the ared the and and of the the the the the the the the the the the the the the the the serer the the the the the the the the the the the the the the the the the the the the the the the the the the t
----- temperature: 0.5
----- Generating with seed: "ou not yet understand?" he paused. "we m"
ou not yet understand?" he paused. "we mane the the sid he foutare the wire thang the teand on the eabe are the sering the weoute tous int of the pouched he paling surithe the bece tit on the the tial 

#fit the model for more 20 epochs

In [30]:

import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Fit the model
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=20,
          callbacks=[print_callback])

Epoch 1/20
2156/2163 [============================>.] - ETA: 0s - loss: 2.4984******************************************************
----- Generating text after Epoch: 0
----- temperature: 0.2
----- Generating with seed: "an attractive young woman always drew ey"
an attractive young woman always drew ey his and was wand the bed and the the the the cout an the the core the sored an the bered an the cout in the the the the his and and on the the the corat he the the mong the bering of the the the his and and and of the the the sous the toring the bere the the the her bered and an the the the the the the her the bered and and the the the the has and an the her the bede the the the her and and he th
----- temperature: 0.5
----- Generating with seed: "an attractive young woman always drew ey"
an attractive young woman always drew ey his and ber tiy lang of the the his sount ans an doon thit andery foul the soung the sting ond no ho had the rose wat doce to his long ou the the burter sot to 

#fit the model for more 30 epochs

In [29]:

import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Fit the model
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=30,
          callbacks=[print_callback])

Epoch 1/30
2155/2163 [============================>.] - ETA: 0s - loss: 2.4979******************************************************
----- Generating text after Epoch: 0
----- temperature: 0.2
----- Generating with seed: " is lieutenant jerome collet. direction "
 is lieutenant jerome collet. direction the the cored the the sound the wang the the the the the the the the ware the wing the the the dound the cored and the the the the the the the care the sing the the gang the sound the the the and and and the here the sered the cared the the gared the ware the the cound and in the ware the the toun the caring the grong the pher the dong the the her the the the the the the the the the the sing the t
----- temperature: 1.0
----- Generating with seed: " is lieutenant jerome collet. direction "
 is lieutenant jerome collet. direction saawe'y triwg, en myou eang to
bleanlmigra de ternt on th hal bentin pone wisser amcsere coulld, hi hefus sa bang.
on shed honed erthcico tungera
deres, hou wof 

In [30]:
model.save("model.h5")

#fit the model for more 50 epochs

In [33]:
from keras.models import load_model
model = load_model('/content/model (2).h5')

In [34]:

import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Fit the model
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback])

Epoch 1/50
2156/2163 [============================>.] - ETA: 0s - loss: 1.4156******************************************************
----- Generating text after Epoch: 0
----- temperature: 1.0
----- Generating with seed: "me that."

                             "
me that."

                                  

b!camet, she levened. "many clad atico.

4hes marreal guering da tatneging to do floot, and she inext, you new it reluer memouncieses of any, devin to are almomais. "p." sip
simplwien mystep into the for uncolked univer in-xift.
that's the saffty. open her had been leigh upped his fordee of to think. "my elice is not ad
himspert," canved, she was arrip knight saw
youghting to you inteat fi
2163/2163 [==============================] - 50s 18ms/step - loss: 1.4157
Epoch 2/50
2163/2163 [==============================] - ETA: 0s - loss: 1.4090******************************************************
----- Generating text after Epoch: 1
----- temperature: 1.0
----- Generating with seed: 

#fit the model for more 100 epochs

In [31]:
from keras.models import load_model
model = load_model('/content/modlo.h5')

In [ ]:

import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Fit the model
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=100,
          callbacks=[print_callback])

Epoch 1/100
2163/2163 [==============================] - ETA: 0s - loss: 1.2227******************************************************
----- Generating text after Epoch: 0
----- temperature: 1.0
----- Generating with seed: "tared blankly into the
void, trying to p"
tared blankly into the
void, trying to prive or a mardience, sounding made nee--close claude droockment as myshood.
placing him. fache was recele me the symbolism and the time tyster intention. the city. finally. the meached his reviling his eatter hid finiler, he destined the coclect with the fifteling up to tiston's secrated lang. even lifed the facal serfo too glass humbely off. nothoring backs, and getting history. "although langdon
2163/2163 [==============================] - 48s 21ms/step - loss: 1.2227
Epoch 2/100
2163/2163 [==============================] - ETA: 0s - loss: 1.2203******************************************************
----- Generating text after Epoch: 1
----- temperature: 1.0
----- Generating with seed

### start with high level modeling

In [3]:
!pip install transformers[torch]>=4.9.0

In [ ]:
data=str(data)

In [ ]:
with open("/content/drive/MyDrive/data_brown.txt", "w") as f:
    f.write(data)

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [ ]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

In [ ]:
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [ ]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  trainer.train()
  trainer.save_model()


In [ ]:
train_file_path ="/content/drive/MyDrive/data_brown.txt"
output_dir = '/content/drive/MyDrive/Colab Notebooks/custom_gpt'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 50.0
save_steps = 50000
model_name = 'gpt2'

In [ ]:
# Train
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,5.840400
1000,4.819800
1500,4.010000
2000,3.291100
2500,2.688700
3000,2.208700
3500,1.851600
4000,1.607200


Step,Training Loss
500,5.840400
1000,4.819800
1500,4.010000
2000,3.291100
2500,2.688700
3000,2.208700
3500,1.851600
4000,1.607200
4500,1.450300
5000,1.368200


In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [ ]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(model_path, sequence, max_length):

    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))


In [ ]:
# for generate the text
model1_path = "/content/drive/MyDrive/Colab Notebooks/custom_gpt" # the path of the model after save it (ali)
sequence1 = "open knoll descending long slope moving farther awayestate seemed heading toward jagged silhouette wooded land distancelangdon cradling keystone turned passenger seat eyed teabing "
max_len = 500
generate_text(model1_path, sequence1, max_len)

open knoll descending long slope moving farther awayestate seemed heading toward jagged silhouette wooded land distancelangdon cradling keystone turned passenger seat eyed teabing vernet driverconcerned tone driver replied vernet gave quiet smile know musttrail continues long waylangdon felt lucky break ground safely tube station despite heavy damage rain alreadyknights teabing seated alcove near curb let wheel teabing climbed backadvanced several steps back imposing figure emergeddark fully armored automobile far side stationthough vernet seemed lost silent gazed across cab across polished facade towardlangdon sophie stood beside imposing statue glass partition stared back sea of redcollage metal langdon felt shiver apprehension woman visage looked exactly projected visage visage faceperfect straight ahead silhouettesomeone else mirage lay front passenger seat leigh teabing sophie staring straight aheadtoward jagged silhouette buildinglangdon felt shiver apprehension woman visage look